In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import backend as K
from tensorflow.keras import Model

import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 

import sys

In [30]:
#read in data from example_failure_data_sets and choose sheet SYS1
df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR3", engine="openpyxl")
df2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name="CSR1", engine="openpyxl")

In [31]:
# df3 = pd.read_csv("./data/generated_data_soft_rel.csv")
# df3 = pd.DataFrame(df3["a=1000,b=0.0005"])
# l = []
# for x in df3[df3.keys()[0]]:
#     l.append(int(x))
    
# t = [x for x in range(l[-1] + 1)]
# fn = []
# i = 0
# for x in range(l[-1] + 1):
#     if x >= l[i]:
#         i +=1
#     fn.append(i)
    
# df_contrived = pd.DataFrame()
# df_contrived["FN"] = fn
# df_contrived["FT"] = t

In [32]:

# df = df_contrived

In [33]:
#make train and test splits.
#change split fraction to be the ratio of testing data ex, 20/100 for 80/20 train/test
split_fraction = 10/100

(x_train, y_train), (x_test, y_test) = (df["FT"][:len(df["FT"]) - int(len(df["FT"]) * split_fraction)], df["FN"][:len(df["FT"]) - int(len(df["FT"]) *split_fraction)]), (df["FT"][len(df["FT"]) - int(len(df["FT"]) * split_fraction):], df["FN"][len(df["FT"]) - int(len(df["FT"])* split_fraction):])



In [34]:
#model being customized. Change activation, number of layers, layer size. Check names of layers and make sure they 
#are being added correctly in "call". x=layer(x) is the correct way. Always normalize. NN don't like unnormalized

b_values = list()
class MyModel(Model):
    #initialize model with parameter and activation methods.
    def __init__(self, nodes = [60], activations= ["elu"], data = None):
        super(MyModel, self).__init__()
        self.b_flag = False
        self.a = 1
        self.d_in = preprocessing.Normalization() #have to normalize so data centered at 0 and gradient works
        self.d_in.adapt(data)
        self.d = []
        for i in range(len(nodes)):
            if activations[i] ==  "x":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.x_activation))

            elif activations[i] ==  "e":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.e_activation))

            elif activations[i] == "1":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.minus_activation))

            elif activations[i] == "go":
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=self.go_activation))
                self.b_flag = True
            else:
                self.d.append(tf.keras.layers.Dense(nodes[i], activation=activations[i]))
            
        self.d_out = tf.keras.layers.Dense(1,activation=tf.keras.activations.linear) #output layer
        if self.b_flag:
            self.b = tf.Variable(0.01)
    
    
    #custom activations
    def e_activation(self, x):
        return K.exp(x)
    
    def minus_activation(self, x):
        return 1-K.switch(x>0, x, x)
    
    def x_activation(self, x):
        return K.switch(x>0, x, x)

    def go_activation(self, x):
        return self.a * (1-K.exp(-self.b * x) )
    
    def weibull_activation(self,x):
        return self.a *(1 - K.exp(-self.b * K.pow(x,self.c)))

    def dss_activation(self, x): #delayed s shaped
        return self.a * (1 - K.exp(-self.b*x) * (1 + self.b * x))

    def iss_activation(self, x): #inflection s shape
        return (self.a*(1-K.exp(-self.b*x)))/(1+self.c*K.exp(-self.b*x)) #ISS
        
        
    #used to initialize the network. Required method from tensorflow. 
    def call(self, x, **kwargs):
        if self.b_flag:
            b_values.append(self.b.numpy())
        x = self.d_in(x)
        for i in range(len(self.d)):
            x = self.d[i](x)
        x = self.d_out(x)
        return x

In [44]:
class NN_Bagging():
    
    def __init__(self, num_models = 10, lr = 0.1, activations = ["elu"], num_nodes = [50], min_delta=0.0001, patience=10, data = None, loss = "mean_squared_error", num_layers = 1, tol = 25, pmse_tol = 10):
        self.models = [MyModel(num_nodes, activations, data)]*num_models
        self.activation = activations[0]
        self.num_models = num_models
        self.num_nodes = num_nodes[-1]
        self.num_layers = num_layers
        self.lr = lr
        self.delta = min_delta
        self.patience = patience
        self.tol = tol
        self.pmse_tol = pmse_tol
        self.loss = loss
        self.is_fit = False
        for i in range(0,self.num_models):
            self.models[i].compile(loss=loss, optimizer=tf.keras.optimizers.Adam(self.lr), run_eagerly=True)


    #modified fit method for incremental learning. 
    def fit(self, x, y, increment = 0.1, known_size = 0.1, scratch = False):
        print("fitting")
        #calculate split sizes. 
        assert known_size >= increment
        split_start = known_size - increment
        split_end = known_size
        if (not self.is_fit) or scratch: #model hasnt been fit yet so fit to initial data
            for i in range(0,self.num_models):
                self.models[i].fit(x[int(len(x)* split_start):int(len(x)* split_end)], y[int(len(x)* split_start):int(len(x)* split_end)], epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=self.delta, patience=self.patience), verbose=0 )
#                 self.models[i].trainable=False
            self.is_fit = True
        else:
#             if self.mse(self.predict(x[int(len(x)* split_start):int(len(x)* split_end)]), y[int(len(x)* split_start):int(len(x)* split_end)]) > self.pmse_tol:
#                 self._retrain(x, y, increment, known_size)
        
            #copy network and add a layer
            self.num_layers = len(self.models[0].layers) + 1
            for i in range(0,self.num_models):
                ls = []
                acts = []
                for l in range(1, len(self.models[i].layers)):
                    ls.append(model.models[i].layers[l].get_config()["units"])
                    acts.append(model.models[i].layers[l].get_config()["activation"])
#                 print(ls)
                mod = MyModel(ls + [self.num_nodes], acts + [self.activation] , x)
                mod.build((1,))

                for l in range(0, len(self.models[i].layers)): #need to make sure only hidden layers are copied
                    mod.layers[l].set_weights(self.models[i].layers[l].get_weights())
                    mod.layers[l].trainable=False
                self.models[i] = mod
                self.models[i].compile(loss=self.loss, optimizer=tf.keras.optimizers.Adam(self.lr), run_eagerly=True)
 
            #Train on new data and check condition until error is low enough. If never low enough scrap model and just train on all data available
            count = 0
            while(count < 1):
                for i in range(0,self.num_models):
#                     for l in range(0, len(self.models[i].layers)):
#                         print(self.models[i].layers[l].trainable)
                    self.models[i].fit(x[int(len(x)* split_start):int(len(x)* split_end)], y[int(len(x)* split_start):int(len(x)* split_end)], epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=self.delta, patience=self.patience), verbose=0 )
                res = self.mse(self.predict(x[:int(len(x)* split_end)]), y[:int(len(x)* split_end)])
                if res < self.tol:
                    break
                else:
                    break
                    count=count+1
                    split_start = split_start - increment
                    
                    if split_start < 0:
                        print("resetting network")
                        print(x[:int(len(x)* split_end)])
                        for i in range(0,self.num_models):
                            self.models[i] = MyModel([self.num_nodes], [self.activation], x)
                            self.models[i].compile(loss=self.loss, optimizer=tf.keras.optimizers.Adam(self.lr), run_eagerly=True)
                            self.models[i].fit(x[:int(len(x)* split_end)], y[:int(len(x)* split_end)], epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=self.delta, patience=self.patience), verbose=0 )
                            for l in range(0, len(self.models[i].layers) -1): #need to make sure only hidden layers are copied
                                mod.layers[l].trainable=False
                        break
            #freeze current model weights
            for i in range(0,self.num_models):
                self.models[i].trainable = False
            print("current ", count)
        
    def _retrain(self, x, y, increment = 0.1, known_size = 0.1):
        split_start = known_size - increment
        split_end = known_size
        for m in self.models:
            m.layers[-1].trainable=True
            m.layers[-2].trainable=True
        count = 0
        while(True):
            for i in range(0,self.num_models):
                self.models[i].fit(x[int(len(x)* split_start):int(len(x)* split_end)], y[int(len(x)* split_start):int(len(x)* split_end)], epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=self.delta, patience=self.patience), verbose=0 )
            
            res = self.mse(self.predict(x[:int(len(x)* split_end)]), y[:int(len(x)* split_end)])
            if res < self.pmse_tol:
                break
            else:
                count=count+1
                split_start = split_start - increment

                if split_start < 0 or count> 0:
                    print("resetting network")
                    for i in range(0,self.num_models):
                        self.models[i] = MyModel([self.num_nodes], [self.activation], x)
                        self.models[i].compile(loss=self.loss, optimizer=tf.keras.optimizers.Adam(self.lr), run_eagerly=True)
                        self.models[i].fit(x[:int(len(x)* split_end)], y[:int(len(x)* split_end)], epochs=3500, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=self.delta, patience=self.patience), verbose=0 )
                    break
        for i in range(0,self.num_models):
                self.models[i].trainable = False
        print("backtracking ", count)
    
    #basic fit that fits on just a small set of data that is new. 
    def train_on_batch(self, x, y, loss = "mean_squared_error"):
        for i in range(0,self.num_models):
            self.models[i].train_on_batch(x, y)


    #inputs x in to model and outputs the result of the model for each x
    def predict(self, x):
        y_res = [0]*len(x)
        y = None
        for m in self.models:
            y = m.predict(x) #y is what the model predicts
            for k in range(0,len(y)):
                y_res[k] += y[k][0]/self.num_models
        return y_res
    
    #used internally to get MSE as error metric. 
    def mse(self, data, actual):  
        mse =0;
        for a, b in zip(data, list(actual)):
    #         print(a,b)
            mse += (a-b)**2
        #     print("{}, {}, {}".format( a, b,(a-b)**2))

        return mse/len(data)
        

In [36]:

# model = NN_Bagging(lr=0.01, num_nodes=[50], data=x_train, min_delta=0.0001, activations=['elu'], num_layers = 1)

# model.fit(x_train, y_train)

In [37]:

import time

In [46]:
fractions = [10, 20, 30, 40, 50, 60, 70, 80, 90]
y_results = {}
# fractions = [20, 40, 60, 80]
# fractions = [30, 60, 90]
# fractions = [50]
lr = 0.1
nodes = 60
activations = ["elu"]#, "relu", "go", "sigmoid", "tanh"]
t_start = time.time()
datasets = [ ("CSR3", "CSR1"), ("CSR3", "CSR2"), ("CSR3", "S2"), ("S2", "CSR1"), ("S2", "CSR3"), ("SYS2", "CSR1"), ("SYS2", "CSR3")  ]

a = "elu"

# model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=["tanh"])
# model.fit(df["FT"], df["FN"])
for d in datasets:
    y_results[d] = pd.DataFrame()
    df = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d[0], engine="openpyxl")
    df2 = pd.read_excel("./data/example_failure_data_sets.xlsx", sheet_name=d[1], engine="openpyxl")
    model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.001, activations=[a], tol=40, pmse_tol = 40)
    
#     model.fit(df["FT"], df["FN"],increment = 1, known_size = 1)
    for j in range(len(fractions)):
        split_fraction = fractions[j]/100
#         model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=[a], num_layers = 1)
#         x_train, y_train = pd.Series(list(df2["FT"][:int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][:int(len(df2["FT"]) * split_fraction)]))
#         x_train, y_train = pd.Series(list(df2["FT"][int(len(df2["FT"]) * (split_fraction - fractions[0]/100)):int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * (split_fraction -fractions[0]/100)):int(len(df2["FT"]) * split_fraction)]))
#         x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))]))
    
#         model.fit(df["FT"], df["FN"], increment = fractions[0]/100, known_size = fractions[j]/100, scratch = True)
#         model.fit(df2["FT"], df2["FN"], increment = fractions[0]/100, known_size = fractions[j]/100)
        if j % 3 == 0:
            model = NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.001, activations=[a], tol=20, pmse_tol = 20)
            model.fit(df["FT"], df["FN"],increment = 1, known_size = 1, scratch=True)
            model.fit(df2["FT"], df2["FN"], increment = fractions[j]/100, known_size = fractions[j]/100, scratch=False)
        else:
            model.fit(df2["FT"], df2["FN"], increment = fractions[0]/100, known_size = fractions[j]/100, scratch=False)

        y_results[d]["split:{}step:{}".format(fractions[j], fractions[0])] = pd.Series(model.predict(df2["FT"]))
t_end = time.time()
print(t_end - t_start)
# #elu: 55 nodes
# #go: 60 nodes
with pd.ExcelWriter("./results/transfer_results_incremental.xlsx") as ex:
    for i in y_results.keys():
        y_results[i].to_excel(ex, "{}, {}".format(i[0], i[1]), index=False)


fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  0
fitting
current  0
fitting
current  0
fitting
fitting
current  

In [47]:
with pd.ExcelWriter("./results/transfer_results_incremental.xlsx") as ex:
    for i in y_results.keys():
        y_results[i].to_excel(ex, "{}, {}".format(i[0], i[1]), index=False)



In [ ]:
activations = ["elu", "relu", "go", "sigmoid", "tanh"]

t_start = time.time()
for a in activations:
    for j in range(int(len(df) - 0.2 * len(df)), len(df)):  
    #     (x_train, y_train), (x_test, y_test) = (df2["FT"][:j], df2["FN"][:j]), (df2["FT"][j:], df2["FN"][j:])
        (x_train, y_train), (x_test, y_test) = (df["FT"][:j], df["FN"][:j]), (df["FT"][j:], df["FN"][j:])
    #     print(x_train)
        model = NN_Bagging(data=x_train, min_delta=0.0001,lr=0.1, num_nodes=[60], activations=[a])
        model.fit(x_train, y_train)
        y_result["sys1_{}_{}".format(a, j)] = pd.Series(model.predict(df["FT"]))
print(y_result)

t_end = time.time()



In [ ]:
y_result.to_csv("./results/sys1_small_step_results.csv")

In [ ]:
# fit to the training data with set number of epochs
# fractions = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# t_start = time.time()

# split_fraction = 20/100
# # (x_train, y_train), (x_test, y_test) = (df2["FT"][:len(df2["FT"]) - int(len(df2["FT"]) * split_fraction)], df2["FN"][:len(df2["FT"]) - int(len(df2["FT"]) *split_fraction)]), (df2["FT"][len(df2["FT"]) - int(len(df2["FT"]) * split_fraction):], df["FN"][len(df2["FT"]) - int(len(df2["FT"])* split_fraction):])


# (x_train, y_train), (x_test, y_test) = (df["FT"][:len(df["FT"]) - int(len(df["FT"]) * split_fraction)], df["FN"][:len(df["FT"]) - int(len(df["FT"]) *split_fraction)]), (df["FT"][len(df["FT"]) - int(len(df["FT"]) * split_fraction):], df["FN"][len(df["FT"]) - int(len(df["FT"])* split_fraction):])
# # for j in range(len(fractions)):  
# #     x_train, y_train = pd.Series(list(df2["FT"][:int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][:int(len(df2["FT"]) * split_fraction)]))
# #     x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))]))


# model = NN_Bagging(data=x_train, min_delta=0.001,lr=0.0001, num_nodes=[128, 32], activations=["e", "1"])
# model.fit(x_train, y_train)
# y_result["sys1_80_20"] = pd.Series(model.predict(df["FT"]))
# print(y_result)

# t_end = time.time()





In [ ]:
# x_train, y_train = df2["FT"][int(len(df2["FT"] )* (split_fraction-10/100)):int(len(df2["FT"] )* split_fraction)], df2["FN"][int(len(df2["FT"] )* (split_fraction-10/100)):int(len(df2["FT"]) * split_fraction)]
# y_result = pd.DataFrame()
y_result.to_csv("./results/dlb_results.csv")
y_result


In [ ]:
# fractions = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# # fractions = [20, 40, 60, 80]
# # fractions = [30, 60, 90]
# # fractions = [50]
# lr = 0.1
# nodes = 55
# y_result = pd.DataFrame()
# models = []
# t_start = time.time()


# models[-1].fit(df["FT"], df["FN"])
# for j in range(len(fractions)):
#     split_fraction = fractions[j]/100
# #     x_train, y_train = pd.Series(list(df2["FT"][int(len(df2["FT"])* (split_fraction-fractions[0]/100)):int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][int(len(df2["FT"] )* (split_fraction-fractions[0]/100)):int(len(df2["FT"]) * split_fraction)]))
# #     x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))]))
#     x_train, y_train = pd.Series(list(df2["FT"][int(len(df2["FT"]) * (split_fraction -fractions[0]/100)):int(len(df2["FT"]) * split_fraction)])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * (split_fraction -fractions[0]/100)):int(len(df2["FT"]) * split_fraction)]))
#     x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + 10/100))]))
 
#     print(len(x_test), len(y_test))

#     models.append(NN_Bagging(lr=lr, num_nodes=[nodes], data=x_train, min_delta=0.0001, activations=["go"]))
    
#     print(x_train, y_train, x_test, y_test)
#     models[-1].fit(x_train, y_train)
#     y_result["split:{}".format(fractions[j])] = pd.Series(models[-1].predict(x_test))

# t_end = time.time()

# # #elu: 55 nodes
# # #go: 60 nodes



In [ ]:
# y_result3_layer = y_result
# y_result
# y_result.to_csv("results/tanh_steping_scratch.csv")

In [ ]:
#predictive mean squared error. one of the more relavant statistics. Doesn't tell the whole story as size
#magnitude grows so the later points are going to have a higher penalty. Still error is high
pmse_list = []
for i in range(len(fractions)):    
    pmse =0
    split_fraction = fractions[i]/100
    y_test = pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))]))
#     print(y_test)
    count = 0
    for a, b in zip(y_result[y_result.keys()[i]][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))], y_test):
        if not np.isnan(a):
            count +=1
            pmse += (a-b)**2
#             print("{}, {}, {}".format( a, b,(a-b)**2))
    pmse = pmse/count
    pmse_list.append(pmse)


In [ ]:
mse_list = []
for i in range(len(lr)* len(num_nodes)):    
    mse =0;
    for a, b in zip(y_result[y_result.keys()[i]][int(0.8* len(y_result[y_result.keys()[i]])):], list(y_test)):
#         print(a,b)
        mse += (a-b)**2
    #     print("{}, {}, {}".format( a, b,(a-b)**2))

    print(mse/len(y_train))
    mse_list.append(mse/len(y_train))


In [ ]:
i = -1
split_fraction = fractions[i]/100
x_test, y_test = pd.Series(list(df2["FT"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))])), pd.Series(list(df2["FN"][int(len(df2["FT"]) * split_fraction):int(len(df2["FT"]) * (split_fraction + fractions[0]/100))]))


y_plot = plt.plot(x_test[:-1], y_result[y_result.keys()[i]] , "dimgray", linestyle="--", marker="o", label="predicted")
actual_plot = plt.plot(x_test,  y_test, "blue", linestyle="--", marker="o", label="actual")

# y_result[y_result.keys()[-10]]
# actual = plt.plot(df["FT"], df["FN"], "black", label="sys2")
plt.xlabel("Time")
plt.ylabel("Number of Failures")
plt.legend()
#     plt.savefig("./results/{}_testing_fraction_go.jpg".format(fractions[x]),dpi=300, bbox_inches = "tight" )
plt.show()

In [ ]:
incremental_pmse = pmse_list
# incremental_mse = mse_list
incremental_results = y_result

# scratch_pmse = pmse_list
# scratch = y_result
# scratch_mse = mse_list

In [ ]:
from matplotlib import lines
from matplotlib import markers
linestyles = list(lines.lineStyles.keys())
markers = list(markers.MarkerStyle.markers.keys())
go = plt.plot(df["FT"],  y_result[y_result.keys()[0]], "dimgray", linestyle=linestyles[0], marker=markers[0], label="go")
one = plt.plot(df["FT"], y_result[y_result.keys()[1]], "blue", marker=markers[6],linestyle=linestyles[1], label = "elu")
two = plt.plot(df["FT"], y_result[y_result.keys()[2]], "red", marker=markers[2],linestyle=linestyles[2], label = activations[2])
three = plt.plot(df["FT"], y_result[y_result.keys()[3]], "green", marker=markers[3],linestyle=linestyles[3], label = activations[3])
four = plt.plot(df["FT"], y_result[y_result.keys()[4]], "silver", marker=markers[4],linestyle=linestyles[1], label = activations[4])

actual = plt.plot(df["FT"], df["FN"], "black", linestyle=linestyles[0], label="sys1")
plt.xlabel("Time")
plt.ylabel("Number of Failures")
plt.legend()
# plt.savefig("sys2_80_20_activations.jpg",dpi=300, bbox_inches = "tight")